In [ ]:
import cv2
import pandas as pd
import numpy as np

In [ ]:
frame_folder = "DEFINE FRAME FOLDER PATH"
stacked_index_output = frame_folder + "\stacked_index.csv"

In [ ]:
frame_index = frame_folder + r"\frame_index.csv"
train_set_index = frame_folder + r"\train_index.csv"
test_set_index = frame_folder + r"\test_index.csv"
validation_set_index = frame_folder + r"\validation_index.csv"

In [ ]:
frames = pd.read_csv(frame_index)
train_set = pd.read_csv(train_set_index)
test_set = pd.read_csv(test_set_index)
validation_set = pd.read_csv(validation_set_index)

In [ ]:
frames

In [ ]:
train_set

In [ ]:
test_set

In [ ]:
validation_set

In [ ]:
complete_set = pd.concat([train_set, test_set, validation_set])
complete_set

In [ ]:
def _load_images(data):
    
    frames = data["Frame"].values
    
    loaded_frames = list()
    
    for frame in frames:
        
        current_path = "%s\\%s" % (frame_folder, frame)
        
        current_frame = cv2.imread(current_path)
        loaded_frames.append(current_frame)
    
    return np.array(loaded_frames)
    
    
def _image_path(data):
    
    any_frame = str(data["Frame"].values[0])
    
    video_prefix_end_index = any_frame.index("_f")
    video_prefix = any_frame[0:video_prefix_end_index]  
    
    timestamp = data["Timestamp_no_millis"].iloc[0]
    
    stacked_frame_path = "%s\\%s_stacked_v2_%s.jpg" % (frame_folder, video_prefix, timestamp)
    
    return stacked_frame_path
    

def _reduce_data_frame(data, image_path):
    
    return pd.DataFrame({
        "Timestamp_no_millis" : [data["Timestamp_no_millis"].iloc[0]],
        "Velocity" : [data["velocity"].iloc[0]],
        "Waterlevel [mm]" : [data["Waterlevel [mm]"].iloc[0]],
        "Percentage Full [%%]" : [data["Percentage Full [%%]"].iloc[0]],
        "Frame" : [image_path]
    })
        

def _stack_images(data):
        
    frames = _load_images(data)
    
    n = len(frames)
    
    if n < 25:
        return None
    
    frames = frames[-25:]
    
    if len(frames) != 25:
        aslkdjkajsd
    
    stacked_frames = np.concatenate(frames, axis=0) # vertical concat.
    
    image_path = _image_path(data)
    
    cv2.imwrite(image_path, stacked_frames)
    
    return _reduce_data_frame(data, image_path)


def stack_data_set(data, frames):
    data_merged_with_frames = frames.merge(data, on="Timestamp_no_millis")
    stacked_data = data_merged_with_frames.groupby("Timestamp_no_millis").apply(_stack_images)
    
    return stacked_data

In [ ]:
training_set_stacked = stack_data_set(complete_set, frames)

In [ ]:
training_set_stacked

In [ ]:
training_set_stacked.to_csv(stacked_index_output)

In [ ]:
stacked_index_output